In [2]:
import pandas as pd
import numpy as np

In [6]:
df_test = pd.read_csv('test_off.csv')

In [7]:
df_test[df_test['session_id'] == '1d688ec168932']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
5,004A07DM0IDW,1d688ec168932,1541555792,6,clickout item,3241426,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [3]:
def explode_position_scalable(df_in, col_expl, pipe='|'):
    """Explode column col_expl of array type into multiple rows."""
    
    n = 60000  #chunk row size
    list_df = [df_in[i:i+n] for i in range(0,df_in.shape[0],n)]
    # Handle the first alone to have the correct headers
    df_x = explode_position(list_df[0], col_expl, flag_conversion=False, pipe_type=pipe)
    df_x = df_x.rename(columns={col_expl:'item_id'})
    df_x.to_csv('exploded.csv', index=False)
    list_df.pop(0)
    for df_x in list_df:
        df_x = explode_position(df_x, col_expl, flag_conversion=False, pipe_type=pipe)
        df_x = df_x.rename(columns={col_expl:'item_id'})
        df_x.to_csv('exploded.csv', mode='a', header=False, index=False)

    df_out = pd.read_csv('exploded.csv')

    os.remove('exploded.csv')
    return df_out
  
def explode_position(df_in, col_expl, flag_conversion = True, pipe_type='|'):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    if(pipe_type== '|'):
        df.loc[:, col_expl] = df[col_expl].apply(string_to_array)
    elif(pipe_type==' '):
        df.loc[:, col_expl] = df[col_expl].apply(space_to_array)
    df.loc[:, 'position'] = df[col_expl].apply(get_position)
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_expl].str.len())
         for col in df.columns.drop(col_expl)}
    )
    df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
    df_out.loc[:, 'position'] = np.concatenate(df['position'].values)
    if(flag_conversion):
        df_out.loc[:, col_expl] = df_out[col_expl].apply(int)

    return df_out
  
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out
  
def get_position(l):
    pos = []
    for i in range(0,len(l)):
        pos.append(i)
    return pos

def create_recent_index(df_orig, grouped=False):
    df_list_int = df_orig.groupby('session_id').apply(lambda x: get_list_session_interactions(x)).reset_index(name='all_interactions')
    df_list_int = df_list_int[['session_id', 'all_interactions']]
    if(grouped):
        return df_list_int
    df_orig = (df_orig.merge(df_list_int, left_on=['session_id'], right_on=['session_id'], how="left"))
    #del df_orig['all_interactions']
    return df_orig
  
def get_list_session_interactions(group):
    group = group[~group['reference'].isnull()]
    group.loc[:,'reference'] = group['reference'].apply(str)
    list_values = group.reference.drop_duplicates()
    joined = " ".join(list_values)
    return joined

def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [4]:
def get_clickouts(df_test):
    df_test['step_max'] = df_test.groupby(['user_id'])['step'].transform(max)
    df_clickout = df_test[(df_test['step_max'] == df_test['step']) & (df_test['action_type'] == 'clickout item')]
    del df_clickout['step_max']
    return df_clickout

In [62]:
mask = (df_test["action_type"] == "clickout item") | (df_test["action_type"] == "interaction item rating") | (df_test["action_type"] == "interaction item image") | (df_test["action_type"] == "interaction item deals")
df_test = df_test[mask]

In [63]:
df_recent = create_recent_index(df_test)
df_recent

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,all_interactions
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,2059240 1050068 3241426
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,2059240 1050068 3241426
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,2059240 1050068 3241426
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,2059240 1050068 3241426
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,2059240 1050068 3241426
5,004A07DM0IDW,1d688ec168932,1541555792,6,clickout item,3241426,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,2059240 1050068 3241426
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,2059240 1050068 3241426
7,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28,
8,00Y1Z24X8084,26b6d294d66e7,1541651766,1,clickout item,3843244,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,3843244
9,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,3843244


In [64]:
def complete_prediction(df_test_nation):
    df_test_nation['item_recommendations'] = df_test_nation.apply(lambda x: fill_recs(x.impressions), axis=1)
    #print('No prediction for #' + str(df_missed.shape[0]) + 'items')
    df_out_nation = df_test_nation[['user_id', 'session_id', 'timestamp','step', 'item_recommendations']]
    return df_out_nation

def list_to_space_string(l):
    """Return a space separated string from a list"""
    s = " ".join(l)
    return s

def fill_recs(imp):
    l = imp.split('|')
    return list_to_space_string(l)

In [74]:
def complete_prediction_recent(df):
    df['item_recommendations'] = df.apply(lambda x: concat_interactions(x), axis=1)
    df_out = df[['user_id', 'session_id', 'timestamp','step', 'item_recommendations']]
    return df_out

def concat_interactions(row):
    spl_int = row.all_interactions.split(' ')
    spl_int = spl_int[::-1]
    spl_imp = row.impressions.split('|')
    result = []
    for i in spl_int:
        result.append(str(i))
    for i in spl_imp:
        if(str(i) not in result):
            result.append(str(i))
    return " ".join(result)

In [66]:
df_sub = get_submission_target(df_recent)

In [76]:
df_predicted = df_sub[df_sub['all_interactions'] != '']
df_not_predicted = df_sub[df_sub['all_interactions'] == '']

In [77]:
df_not_predicted = complete_prediction(df_not_predicted)
df_not_predicted.head()

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,session_id,timestamp,step,item_recommendations
7,009RGHI3G9A3,f05ab0de907e2,1541570940,2,10884872 7065316
285,05BAHQHA5RCI,9f279162e8d5c,1541633993,5,2755290 14054 10243166 3498770 726906 2616752 ...
391,09X618WIVTP3,b1935d2a43d5e,1541688108,3,1120687 2165594 5763948 519501 595286 3386520 ...
438,0BZN2KFMHLZF,d9497eeb946dc,1541668916,1,39730 133101 39758 7029236 455086 3887112 1070...
479,0D1XFBRWUX7Z,35e873299a45f,1541586069,1,54865 54803 54953 14211 95510 54838 54794 1484...


In [78]:
df_predicted = complete_prediction_recent(df_predicted)
df_predicted

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,session_id,timestamp,step,item_recommendations
6,004A07DM0IDW,1d688ec168932,1541555799,7,3241426 1050068 2059240 2033381 1724779 127131...
9,00Y1Z24X8084,26b6d294d66e7,1541651823,2,3843244 2714480 4476010 3833012 9017890 198100...
11,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,2817590 3565720 2947584 4115018 2039671 383653...
50,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,1177554 693591 1177542 6639820 7304020 749491 ...
94,0339C84S24ET,89171d441a304,1541615683,36,37717 8160620 4957186 13361 5647680 116764 898...
96,0386OH8JDE1Q,e09591d07cdef,1541620536,2,5488246 1193320 3858774 4552034 10620372 22696...
98,03LTH89QY623,7663406cf586c,1541554183,4,59566 241961 906477 991561 353701 1149665 7725...
120,03VT0ODUTZB0,725e8adf70e86,1541632490,23,322491 1070666 109938 164193 632366 1362450 16...
284,03XH0JWCWHAM,73f4c417ff730,1541566143,176,6452022 42618 42858 6298428 3148690 43133 5762...
306,062OG5L9831R,289b34a2bb4dd,1541663105,21,9914686 8474478 5828606 6618430 7828086 922547...


In [3]:
df_out = pd.concat([df_predicted, df_not_predicted], ignore_index=True, sort=False)
df_out.to_csv('sub_rule_based.csv')

NameError: name 'df_predicted' is not defined

In [80]:
df_out.head()

,user_id,session_id,timestamp,step,item_recommendations
0,004A07DM0IDW,1d688ec168932,1541555799,7,3241426 1050068 2059240 2033381 1724779 127131...
1,00Y1Z24X8084,26b6d294d66e7,1541651823,2,3843244 2714480 4476010 3833012 9017890 198100...
2,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,2817590 3565720 2947584 4115018 2039671 383653...
3,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,1177554 693591 1177542 6639820 7304020 749491 ...
4,0339C84S24ET,89171d441a304,1541615683,36,37717 8160620 4957186 13361 5647680 116764 898...


In [54]:
df_out[df_out['user_id'] == '000GO9NY6P4M']

,user_id,session_id,timestamp,step,item_recommendations
248422,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,157710 1287958 160577 1618677 7231396 319866 4...


In [18]:
df_predicted.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,all_interactions
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...,2059240 1050068 3241426
9,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...,3843244
11,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,3565720|2947584|4115018|2039671|3836538|801409...,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...,2817590
50,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1451247|559056|1045096|1963879|693596|1967173|...,80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...,552311 749491 7304020 6639820 1177542 693591 1...
94,0339C84S24ET,89171d441a304,1541615683,36,clickout item,NaN,TR,"Antalya, Turkey",mobile,NaN,13361|5647680|116764|898719|8276346|9168|19325...,185|84|30|19|46|77|123|23|25|25|26|39|73|56|96...,4957186 8160620 37717


In [4]:
df_t = pd.read_csv('sub_rule_based.csv')
df_t.head()

,Unnamed: 0,user_id,session_id,timestamp,step,item_recommendations
0,0,004A07DM0IDW,1d688ec168932,1541555799,7,3241426 1050068 2059240 2033381 1724779 127131...
1,1,00Y1Z24X8084,26b6d294d66e7,1541651823,2,3843244 2714480 4476010 3833012 9017890 198100...
2,2,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,2817590 3565720 2947584 4115018 2039671 383653...
3,3,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,1177554 693591 1177542 6639820 7304020 749491 ...
4,4,0339C84S24ET,89171d441a304,1541615683,36,37717 8160620 4957186 13361 5647680 116764 898...


In [ ]:
df_t[df_t['user_id'] == '004A07DM0IDW']

In [50]:
df_out.shape[0]

253573

In [51]:
df_t.shape[0]

152343

In [56]:
df_t[df_t['user_id'] == '000GO9NY6P4M']

,Unnamed: 0,user_id,session_id,timestamp,step,item_recommendations
248422,248422,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,157710 1287958 160577 1618677 7231396 319866 4...


In [16]:
df_gt = pd.read_csv('gt.csv')
mask = (df_gt["action_type"] == "clickout item") | (df_gt["action_type"] == "interaction item rating") | (df_gt["action_type"] == "interaction item image") | (df_gt["action_type"] == "interaction item deals")
df_gt = df_gt[mask]
df_gt.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
16,03P4VFKK12UO,325fafb5fa450,1541107435,17,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
17,03P4VFKK12UO,325fafb5fa450,1541107435,18,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
18,03P4VFKK12UO,325fafb5fa450,1541107457,19,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
19,03P4VFKK12UO,325fafb5fa450,1541107457,20,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
20,03P4VFKK12UO,325fafb5fa450,1541107467,21,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN


In [17]:
df_gt[df_gt['user_id'] == '03P4VFKK12UO']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
16,03P4VFKK12UO,325fafb5fa450,1541107435,17,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
17,03P4VFKK12UO,325fafb5fa450,1541107435,18,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
18,03P4VFKK12UO,325fafb5fa450,1541107457,19,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
19,03P4VFKK12UO,325fafb5fa450,1541107457,20,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
20,03P4VFKK12UO,325fafb5fa450,1541107467,21,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
21,03P4VFKK12UO,325fafb5fa450,1541107467,22,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
22,03P4VFKK12UO,325fafb5fa450,1541107467,23,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
23,03P4VFKK12UO,325fafb5fa450,1541107467,24,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
24,03P4VFKK12UO,325fafb5fa450,1541107467,25,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
25,03P4VFKK12UO,325fafb5fa450,1541107467,26,interaction item image,65685,US,"Bakersfield, USA",desktop,NaN,NaN,NaN
